#  Introdução ao Python
## Professor: Luiz Ferreira

## Módulo 3

### Python para Mobile

_____________

## Um jogo: Pong

Podemos utilizar o Kivy para desenvolver jogos. O uso do Kivy não é incomum em aplicações já hospedadas nas lojas do Google e Apple. Podemos encontrar versões de jogos famosos desenvolvidos em Kivy, tais como, FlappyBird, 2048, entre muitos outros, podemos encontrar exemplos referenciados pelo próprio projeto Kivy em: https://github.com/kivy/kivy/wiki/List-of-Kivy-Projects.

Começamos a desenvolver nosso jogo:

Devemos criar (novamente) dois arquivos:

- .py
- .kv

Vamos começar pelo arquivo python:

Devemos importar os elementos que vamos utilizar no nosso projeto :

In [ ]:
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.properties import NumericProperty, ReferenceListProperty,ObjectProperty
from kivy.vector import Vector
from kivy.clock import Clock

Aqui vemos os pacotes App e Widget já conhecidos de outros projetos, temos também os pacotes de properties que são os responsáveis por configurações de nossas aplicações. Além das citadas, temos a Vector, responsável por permitir o uso de vetores para posicionar elementos dentro da tela (observe que nesse projeto, não utilizamos o pacote GridLayout) e, temos também, o pacote Clock que permite executar determinadas funções de forma agendada, como um planejamento de execução.

In [ ]:
class PongPaddle(Widget):
    score = NumericProperty(0)

    def bounce_ball(self, ball):
        if self.collide_widget(ball):
            vx, vy = ball.velocity
            offset = (ball.center_y - self.center_y) / (self.height / 2)
            bounced = Vector(-1 * vx, vy)
            vel = bounced * 1.1
            ball.velocity = vel.x, vel.y + offset

Temos a classe PongPaddle, nossas "raquetes", também podemos notar a variável score surgindo como um item a ser acrescido durante a partida. Detro dessa classe temos a função bounce_ball que será a responsável pela movimentação da bola sobre a mesa. Podemos notar que nesse momento, trabalha-se tanto a colisão da bola quanto a velocidade adquirida com esse evento.

In [ ]:
class PongBall(Widget):
    velocity_x = NumericProperty(0)
    velocity_y = NumericProperty(0)
    velocity = ReferenceListProperty(velocity_x, velocity_y)

    def move(self):
        self.pos = Vector(*self.velocity) + self.pos

A segunda classe, chamada de PongBall, tratará da bola em sí, perceba que aqui temos os referenciais da velocidade de movimentação da bola, note também que temos aqui uma função que irá realizar a movimentação da bola sobre a mesa.

In [ ]:
class PongGame(Widget):
    ball = ObjectProperty(None)
    player1 = ObjectProperty(None)
    player2 = ObjectProperty(None)

    def serve_ball(self, vel=(4, 0)):
        self.ball.center = self.center
        self.ball.velocity = vel

    def update(self, dt):
        self.ball.move()

        # bounce of paddles
        self.player1.bounce_ball(self.ball)
        self.player2.bounce_ball(self.ball)

        # bounce ball off bottom or top
        if (self.ball.y < self.y) or (self.ball.top > self.top):
            self.ball.velocity_y *= -1

        # went of to a side to score point?
        if self.ball.x < self.x:
            self.player2.score += 1
            self.serve_ball(vel=(4, 0))
        if self.ball.x > self.width:
            self.player1.score += 1
            self.serve_ball(vel=(-4, 0))

    def on_touch_move(self, touch):
        if touch.x < self.width / 3:
            self.player1.center_y = touch.y
        if touch.x > self.width - self.width / 3:
            self.player2.center_y = touch.y

Na classe PongGame notamos que existe a criação de 3 elementos fundamentais para o jogo, a bola e os players. Depois da instancição dos elementos, temos a função serve_ball que irá fornecer as propriedades para o início do game, posicionando a bola no centro com a velocidade padrão 1.

Temos também a função update que irá atualizar o posicionamento da bola e dos paddles que são nossas "raquetes". Criamos então uma raquete para cada jogador. Depois colocamos os limitadores verticais de nossa mesa, que fará com que, quando a bola atinja os limites inferiores e superiores, ela não saia do jogo. Lembre-se que só somaremos um ponto quando a bola ultrapassar a "raquete" do lado correspondente do jogador adversário.

Para registrarmos o placar, temos duas estruturas de decisão (if), onde, percorrendo o valor do eixo X, quando o mesmo for menor que 0 soma-se ponto para um player, ou quando ele ultrapassar a quantidade de pontos totais do eixo X soma-se ponto para o outro jogador.

Por fim definimos a movimentação dos players, para que ela aconteça somente dentro do eixo y.

In [ ]:
class PongApp(App):
    def build(self):
        game = PongGame()
        game.serve_ball()
        Clock.schedule_interval(game.update, 1.0 / 60.0)
        return game

Com isso, definimos nosso construtor de aplicação.

In [ ]:
if __name__ == '__main__':
    PongApp().run()

Por fim, temos a execução de nosso construtor.

### Vamos para o arquivo .kv

Lembramos que o arquivo .kv determina configurações dos elementos utilizados na aplicação. Portanto teremos:

In [ ]:
<PongBall>:
    size: 50, 50 
    canvas:
        Ellipse:
            pos: self.pos
            size: self.size    

In [ ]:
<PongPaddle>:
    size: 25, 200
    canvas:
        Rectangle:
            pos:self.pos
            size:self.size

Esse item definirá o formato, tamanho e posicionamento dos players ("raquetes") do jogo.

In [ ]:
<PongGame>:
    ball: pong_ball
    player1: player_left
    player2: player_right
    
    canvas:
        Rectangle:
            pos: self.center_x-5, 0
            size: 10, self.height
    
    Label:
        font_size: 70  
        center_x: root.width / 4
        top: root.top - 50
        text: str(root.player1.score)
        
    Label:
        font_size: 70  
        center_x: root.width * 3 / 4
        top: root.top - 50
        text: str(root.player2.score)
    
    PongBall:
        id: pong_ball
        center: self.parent.center
        
    PongPaddle:
        id: player_left
        x: root.x
        center_y: root.center_y
        
    PongPaddle:
        id: player_right
        x: root.width-self.width
        center_y: root.center_y

Com isso vamos posicionar nossos elementos de forma com que exista um retângulo que será nossa mesa (canvas), uma linha dividindo o "campo" de cada jogador, 